In [1]:
import re


# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [2]:
### Write your code here
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [4]:
# spark = SparkSession.builder.appName('assn2test').getOrCreate()
# spark.stop()
sc = SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/05 04:09:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
type(sc)

pyspark.context.SparkContext

### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [6]:
### Write your code here
book_43 = sc.textFile("./data/43.txt", minPartitions=96)

In [7]:
type(book_43)

pyspark.rdd.RDD

In [8]:
book_43.take(5)

['The Project Gutenberg eBook of The Strange Case Of Dr. Jekyll And Mr. Hyde, by Robert Louis Stevenson',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms']

In [9]:
str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

True

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [10]:
### Write your code here
print("# of lines = {}".format(book_43.count()))

num_lines = len(open("./data/43.txt").readlines())
print("numbe of lines in the text file is {}".format(num_lines))

# of lines = 2935
numbe of lines in the text file is 2935


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [11]:
### Write your code here
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.lower().split()
#     return b.upper().split()


In [12]:
test_line = "This is an example of that contains 234 and a dash-containing number"
clean_split_line(test_line)

['this',
 'is',
 'an',
 'example',
 'of',
 'that',
 'contains',
 'and',
 'a',
 'dash',
 'containing',
 'number']

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [13]:
### Write your code here
words = book_43.flatMap(clean_split_line)
words.count()

29116

### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [14]:
### Write your code here
distinctWords = words.distinct()
count = distinctWords.count()
print("{} distinct words".format(count)) #number of distinct words!

4296 distinct words


### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [15]:
### Write your code here
words_with_ones = words.map(lambda x: (x, 1))
words_with_ones.take(10)

[('the', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('of', 1),
 ('the', 1),
 ('strange', 1),
 ('case', 1),
 ('of', 1),
 ('dr', 1)]

In [16]:
sorted_words_with_ones = words_with_ones.sortByKey()
sorted_words_with_ones.take(10)

[('_dines_', 1),
 ('_is_', 1),
 ('_pede', 1),
 ('_protégé_', 1),
 ('_th', 1),
 ('_to', 1),
 ('_who', 1),
 ('_why_', 1),
 ('_with', 1),
 ('_à', 1)]

In [17]:
book_43_counts = sorted_words_with_ones.reduceByKey(lambda x,y: x+y)

In [18]:
project = book_43_counts.filter(lambda x: "project" in x)

In [19]:
project.take(5)

[('project', 88)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [20]:
### Write your code here
book_43_counts2 = book_43_counts.map(lambda x: (x[1], x[0]))

In [21]:
book_43_counts2.take(10)

[(3, 'ape'),
 (1, 'approaches'),
 (1, 'assaults'),
 (2, 'belief'),
 (1, 'bird'),
 (1, 'civilly'),
 (7, 'common'),
 (1, 'connived'),
 (1, 'destruction'),
 (1, 'disliked')]

In [22]:
book_43_counts_sorted = book_43_counts2.sortByKey(False)

In [23]:
book_43_counts_sorted.take(20)

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he'),
 (378, 'it'),
 (312, 'you'),
 (308, 'my'),
 (301, 'with'),
 (285, 'his'),
 (244, 'had'),
 (203, 'as'),
 (202, 'for'),
 (195, 'this'),
 (193, 'but')]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [24]:
### Write your code here
from sklearn.feature_extraction import _stop_words
stopWords = _stop_words.ENGLISH_STOP_WORDS

In [25]:
type(stopWords)

frozenset

In [26]:
stop_words = list(stopWords)

In [27]:
# A way to find words containing another word, such as "the" in this case
# items = []
# for id, item in enumerate(list1):
#     if "the" in item:
#         items.append(id)

# T = [list1[i] for i in items]
# T

In [28]:
stop_words[:10]

['con',
 'ie',
 'or',
 'should',
 'herself',
 'no',
 'perhaps',
 'anything',
 'even',
 'nobody']

### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [29]:
### Write your code here
book_43_counts_sorted.take(10)

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he')]

In [30]:
# book_43_counts_filtered = book_43_counts_sorted.filter(lambda x:  in x)
book_43_counts_filtered = book_43_counts_sorted.filter(lambda x: x[1] not in stop_words)

In [31]:
book_43_counts_filtered.take(10)

[(156, 's'),
 (131, 'utterson'),
 (130, 'said'),
 (128, 'mr'),
 (105, 'hyde'),
 (105, 'jekyll'),
 (98, 'gutenberg'),
 (88, 'project'),
 (85, 'man'),
 (72, 'lawyer')]

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [32]:
### Write your code here
book_43_counts_filtered.count()

4034

### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [33]:
### Write your code here

def process_RDD(book_path):
    book = sc.textFile(book_path, minPartitions=96)
    words = book.flatMap(clean_split_line)
    words_with_ones = words.map(lambda x: (x, 1))
    sorted_words_with_ones = words_with_ones.sortByKey()
    book_counts = sorted_words_with_ones.reduceByKey(lambda x,y: x+y)
    book_counts2 = book_counts.map(lambda x: (x[1], x[0]))
#     book_43_counts_sorted = book_43_counts2.sortByKey(False) In case it stops working, maybe this is reason
    stopWords = _stop_words.ENGLISH_STOP_WORDS
    stop_words = list(stopWords)
    book_wout_stops = book_counts2.filter(lambda x: x[1] not in stop_words)
    book = book_wout_stops.sortByKey(False) #False means descending I think

    return book



In [34]:
booky = process_RDD("./data/43.txt")

In [35]:
type(booky)

pyspark.rdd.PipelinedRDD

In [36]:
booky.take(10)

[(156, 's'),
 (131, 'utterson'),
 (130, 'said'),
 (128, 'mr'),
 (105, 'hyde'),
 (105, 'jekyll'),
 (98, 'gutenberg'),
 (88, 'project'),
 (85, 'man'),
 (72, 'lawyer')]

In [37]:
#How to create a new RDD from only the word "a"
stuff = booky.filter(lambda x: "the" in x)
print(stuff.take(1))
booky.count()

[]


4034

### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [38]:
### Write your code here
book_84_counts_filtered = process_RDD("./data/84.txt")
book_398_counts_filtered = process_RDD("./data/398.txt")
book_3296_counts_filtered = process_RDD("./data/3296.txt")

In [39]:
print("Book 84 distinct words # = {}".format(book_84_counts_filtered.count()))
print("Book 398 distinct words # = {}".format(book_398_counts_filtered.count()))
print("Book 3296 distinct words # = {}".format(book_3296_counts_filtered.count()))

Book 84 distinct words # = 7016
Book 398 distinct words # = 2421
Book 3296 distinct words # = 7293


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [40]:
### Write your code here
intersect_398_3296 = book_398_counts_filtered.map(lambda x: x[1]).intersection(book_3296_counts_filtered.map(lambda x: x[1]))

intersect_84_398 = book_84_counts_filtered.map(lambda x: x[1]).intersection(book_398_counts_filtered.map(lambda x: x[1]))

print("(intersect_398_3296 count) = {}".format(intersect_398_3296.count()))
print("(intersect_84_398 count) = {}".format(intersect_84_398.count()))

(intersect_398_3296 count) = 1790


(intersect_84_398 count) = 1691


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

#### Write your answer here

* <p style="color: Green">the two tests differ by only 5%.</p>
* <p style="color: Green">We can justify the answer by looking at word count per book. If two books (A and B) have 100 times as many words as the third (book C), [A: 5000wds, B: 5000wds, C: 50wds], books A and B will have more words in common just based on statistics, even if books A and C have much more in common.</p>
* <p style="color: Green">If we are comparing multiple books with roughly the same number of words, then this technique should work ok.</p>

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [41]:
### Write your code here
import numpy as np

book_43_counts_filtered = process_RDD("./data/43.txt")

intersect_43_84_398_3296 = book_398_counts_filtered.map(lambda x: x[1]).intersection(book_3296_counts_filtered.map(lambda x: x[1])).intersection(book_84_counts_filtered.map(lambda x: x[1])).intersection(book_43_counts_filtered.map(lambda x: x[1]))

colCount = intersect_43_84_398_3296.count() #about 1500
colCount

1162

In [42]:
type(intersect_43_84_398_3296.take(10))

list

In [43]:
type(book_43_counts_filtered)

pyspark.rdd.PipelinedRDD

In [44]:
book_43_counts_filtered.take(10)

[(156, 's'),
 (131, 'utterson'),
 (130, 'said'),
 (128, 'mr'),
 (105, 'hyde'),
 (105, 'jekyll'),
 (98, 'gutenberg'),
 (88, 'project'),
 (85, 'man'),
 (72, 'lawyer')]

In [45]:
intersect_43_84_398_3296.take(10)

['belief',
 'indirectly',
 'returns',
 'seek',
 'dust',
 'iron',
 'employee',
 'calculate',
 'volunteer',
 'important']

In [46]:
# book_43_counts_filtered = book_43_counts_sorted.filter(lambda x:  in x)
book_43_counts_filtered = book_43_counts_sorted.filter(lambda x: x[1] not in stop_words)

In [47]:
type(book_43_counts_sorted)
book_43_counts_sorted.take(10)

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he')]

In [48]:
intersection_list = list(intersect_43_84_398_3296.collect())

In [49]:
words_counts_43 = book_43_counts_filtered.filter(lambda x: x[1] in intersection_list)
words_counts_84 = book_84_counts_filtered.filter(lambda x: x[1] in intersection_list)
words_counts_398 = book_398_counts_filtered.filter(lambda x: x[1] in intersection_list)
words_counts_3296 = book_3296_counts_filtered.filter(lambda x: x[1] in intersection_list)



In [50]:
words_counts_43 = words_counts_43.map(lambda x: (x[1], x[0]))

In [51]:
words_counts_84 = words_counts_84.map(lambda x: (x[1], x[0]))

In [52]:
words_counts_398 = words_counts_398.map(lambda x: (x[1], x[0]))

In [53]:
words_counts_3296 = words_counts_3296.map(lambda x: (x[1], x[0]))

In [54]:
words_counts_43 = words_counts_43.sortByKey(False)
words_counts_84 = words_counts_84.sortByKey(False)
words_counts_398 = words_counts_398.sortByKey(False)
words_counts_3296 = words_counts_3296.sortByKey(False)

print(words_counts_43.take(10))
print()
print(words_counts_84.take(10))
print()
print(words_counts_398.take(10))
print()
print(words_counts_3296.take(10))

[('zip', 1), ('younger', 3), ('young', 5), ('years', 11), ('www', 10), ('wrote', 2), ('wrong', 6), ('written', 8), ('writing', 6), ('write', 5)]

[('zip', 1), ('younger', 9), ('young', 30), ('years', 45), ('www', 10), ('wrote', 4), ('wrong', 1), ('written', 7), ('writing', 6), ('write', 11)]

[('zip', 1), ('younger', 2), ('young', 1), ('years', 19), ('www', 6), ('wrote', 1), ('wrong', 3), ('written', 6), ('writing', 4), ('write', 1)]

[('zip', 1), ('younger', 4), ('young', 15), ('years', 51), ('www', 10), ('wrote', 12), ('wrong', 2), ('written', 25), ('writing', 13), ('write', 8)]


In [55]:
counts_matrix = np.zeros([4, colCount])
print(counts_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [56]:
counts_matrix[0, :] = list(words_counts_43.map(lambda x: x[1]).collect())
counts_matrix[1, :] = list(words_counts_84.map(lambda x: x[1]).collect())
counts_matrix[2, :] = list(words_counts_398.map(lambda x: x[1]).collect())
counts_matrix[3, :] = list(words_counts_3296.map(lambda x: x[1]).collect())

<h1 style="color: green">Counts Matrix</h1>

In [57]:
print(counts_matrix)

[[ 1.  3.  5. ...  2.  3.  1.]
 [ 1.  9. 30. ...  4.  9.  1.]
 [ 1.  2.  1. ... 16.  5.  1.]
 [ 1.  4. 15. ...  3. 36. 16.]]


### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [58]:
### Write your code here
from scipy.spatial.distance import euclidean 

<h1 style="color: green">Euclidean Distances</h1>

In [59]:
print(f"The Euclidean distance between book_398 and book_3296 is: {euclidean(counts_matrix[2], counts_matrix[3])}")
print(f"The Euclidean distance between book_84 and book_398 is: {euclidean(counts_matrix[1], counts_matrix[2])}")

The Euclidean distance between book_398 and book_3296 is: 1156.6628722320086
The Euclidean distance between book_84 and book_398 is: 751.6688100486809


#### Write your justification here
* Euclidean distance seems much better than just counting the words based on the larger difference when comparing the three books.

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Write your answer here


## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [60]:
### Write your code here
from pyspark.sql import SparkSession
session = SparkSession(sc)

<h1 style="color: green">Number of entries in file</h1>

In [61]:
%%time

flight_info = session.read.csv("./data/flight_info.csv", header=True, inferSchema=True)
print(flight_info.count())
# flight_info.head(1)

450017
CPU times: user 2.65 ms, sys: 1.9 ms, total: 4.55 ms
Wall time: 4.02 s


### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [62]:
### Write your code here
flight_info.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- WheelsOff: double (nullable = true)
 |-- WheelsOn: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)



In [63]:
uniqueCarrier = flight_info.select('UniqueCarrier').distinct()


<h1 style="color: green">Number of unique airlines</h1>

In [64]:
uniqueCarrier.count()

12

In [65]:
uniqueCarrier.head(20)

[Row(UniqueCarrier='AA'),
 Row(UniqueCarrier='EV'),
 Row(UniqueCarrier='B6'),
 Row(UniqueCarrier='DL'),
 Row(UniqueCarrier='UA'),
 Row(UniqueCarrier='NK'),
 Row(UniqueCarrier='OO'),
 Row(UniqueCarrier='F9'),
 Row(UniqueCarrier='HA'),
 Row(UniqueCarrier='WN'),
 Row(UniqueCarrier='AS'),
 Row(UniqueCarrier='VX')]

### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [66]:
### Write your code here
flight_info.createOrReplaceTempView("Flights")
session.sql('select * from Flights').head(1)

[Row(ID=0, DayOfWeek=2, UniqueCarrier='AA', FlightNum=494, Origin='CLT', Dest='PHX', CRSDepTime=1619, DepTime=1616.0, TaxiOut=17.0, WheelsOff=1633.0, WheelsOn=1837.0, TaxiIn=5.0, CRSArrTime=1856, ArrTime=1842.0, Cancelled=0.0, CancellationCode=None, Distance=1773.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None)]

In [67]:
flight_info.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- WheelsOff: double (nullable = true)
 |-- WheelsOn: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)



In [134]:
flight_delays = session.sql("""
SELECT UniqueCarrier, COUNT(*) 
FROM Flights
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier;
""")

<h1 style="color: green">Flight Delays per Airline</h1>

In [135]:
flight_delays.head(20)

[Row(UniqueCarrier='AA', count(1)=23461),
 Row(UniqueCarrier='EV', count(1)=11596),
 Row(UniqueCarrier='B6', count(1)=9396),
 Row(UniqueCarrier='DL', count(1)=24334),
 Row(UniqueCarrier='UA', count(1)=17701),
 Row(UniqueCarrier='NK', count(1)=4151),
 Row(UniqueCarrier='OO', count(1)=16751),
 Row(UniqueCarrier='F9', count(1)=2988),
 Row(UniqueCarrier='HA', count(1)=1939),
 Row(UniqueCarrier='WN', count(1)=47472),
 Row(UniqueCarrier='AS', count(1)=4488),
 Row(UniqueCarrier='VX', count(1)=2648)]

In [121]:
type(countings)

pyspark.sql.dataframe.DataFrame

### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [70]:
### Write your code here
airlines_info = session.read.csv("./data/airlines.csv", header=False, inferSchema=True)
print(airlines_info.count())
print(airlines_info.head(2))
airlines_info.createOrReplaceTempView("airlines")

6161
[Row(_c0=1, _c1='Private flight', _c2='\\N', _c3='-', _c4='N/A', _c5=None, _c6=None, _c7='Y'), Row(_c0=2, _c1='135 Airways', _c2='\\N', _c3=None, _c4='GNL', _c5='GENERAL', _c6='United States', _c7='N')]


In [71]:
session.sql('select _c1 from airlines').head(1)

[Row(_c1='Private flight')]

In [72]:
session.sql("""
SELECT _c3
FROM airlines;
""").head(5)

[Row(_c3='-'), Row(_c3=None), Row(_c3='1T'), Row(_c3=None), Row(_c3=None)]

In [195]:
flight_delays_fullName = session.sql("""
SELECT UniqueCarrier, first(_c1), COUNT(*)
FROM Flights FF
INNER JOIN airlines AA
on FF.UniqueCarrier = AA._c3
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier;
""")

<h1 style="color: green">Flight Delays per Airline</h1>

In [196]:
flight_delays_fullName.head(20)

[Row(UniqueCarrier='AA', first(_c1)='American Airlines', count(1)=23461),
 Row(UniqueCarrier='AS', first(_c1)='Alaska Airlines', count(1)=4488),
 Row(UniqueCarrier='B6', first(_c1)='JetBlue Airways', count(1)=9396),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count(1)=24334),
 Row(UniqueCarrier='EV', first(_c1)='Atlantic Southeast Airlines', count(1)=11596),
 Row(UniqueCarrier='F9', first(_c1)='Frontier Airlines', count(1)=2988),
 Row(UniqueCarrier='HA', first(_c1)='Hawaiian Airlines', count(1)=1939),
 Row(UniqueCarrier='NK', first(_c1)='Spirit Airlines', count(1)=4151),
 Row(UniqueCarrier='OO', first(_c1)='SkyWest', count(1)=16751),
 Row(UniqueCarrier='UA', first(_c1)='United Airlines', count(1)=17701),
 Row(UniqueCarrier='VX', first(_c1)='Virgin America', count(1)=2648),
 Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count(1)=47472)]

### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [198]:
### Write your code here
session.sql("""
SELECT UniqueCarrier, DayOfWeek, COUNT(*)
FROM Flights
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier, DayOfWeek
ORDER BY UniqueCarrier, DayOfWeek ASC;
""").head(30)

[Row(UniqueCarrier='AA', DayOfWeek=1, count(1)=4639),
 Row(UniqueCarrier='AA', DayOfWeek=2, count(1)=3288),
 Row(UniqueCarrier='AA', DayOfWeek=3, count(1)=2570),
 Row(UniqueCarrier='AA', DayOfWeek=4, count(1)=3142),
 Row(UniqueCarrier='AA', DayOfWeek=5, count(1)=3108),
 Row(UniqueCarrier='AA', DayOfWeek=6, count(1)=2509),
 Row(UniqueCarrier='AA', DayOfWeek=7, count(1)=4205),
 Row(UniqueCarrier='AS', DayOfWeek=1, count(1)=786),
 Row(UniqueCarrier='AS', DayOfWeek=2, count(1)=580),
 Row(UniqueCarrier='AS', DayOfWeek=3, count(1)=573),
 Row(UniqueCarrier='AS', DayOfWeek=4, count(1)=656),
 Row(UniqueCarrier='AS', DayOfWeek=5, count(1)=554),
 Row(UniqueCarrier='AS', DayOfWeek=6, count(1)=470),
 Row(UniqueCarrier='AS', DayOfWeek=7, count(1)=869),
 Row(UniqueCarrier='B6', DayOfWeek=1, count(1)=1754),
 Row(UniqueCarrier='B6', DayOfWeek=2, count(1)=1628),
 Row(UniqueCarrier='B6', DayOfWeek=3, count(1)=1109),
 Row(UniqueCarrier='B6', DayOfWeek=4, count(1)=1101),
 Row(UniqueCarrier='B6', DayOfWeek=

### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [148]:
### Write your code here
Counts = session.sql("""
SELECT UniqueCarrier, first(_c1), COUNT(*)
FROM Flights
INNER JOIN airlines
on Flights.UniqueCarrier = airlines._c3
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier;
""")

Totals = session.sql("""
SELECT UniqueCarrier, COUNT(*) 
FROM Flights
GROUP BY UniqueCarrier;
""")

In [182]:
Counts.take(3)

[Row(UniqueCarrier='AA', first(_c1)='American Airlines', count(1)=23461),
 Row(UniqueCarrier='AS', first(_c1)='Alaska Airlines', count(1)=4488),
 Row(UniqueCarrier='B6', first(_c1)='JetBlue Airways', count(1)=9396)]

In [183]:
Totals.take(3)

[Row(UniqueCarrier='AA', count(1)=73132),
 Row(UniqueCarrier='EV', count(1)=35037),
 Row(UniqueCarrier='B6', count(1)=24602)]

In [151]:
Counts.createOrReplaceTempView("counts")
Totals.createOrReplaceTempView("totals")
session.sql('select * from counts').head(1)

[Row(UniqueCarrier='AA', first(_c1)='American Airlines', count(1)=23461)]

In [203]:
norms = session.sql("""
SELECT c.UniqueCarrier, ROUND(c.`count(1)` / t.`count(1)`,2) AS norm
FROM counts c
INNER JOIN totals t
on c.UniqueCarrier = t.UniqueCarrier;
""")
testing.head(10)

[Row(UniqueCarrier='AA', norm=0.32),
 Row(UniqueCarrier='EV', norm=0.33),
 Row(UniqueCarrier='B6', norm=0.38),
 Row(UniqueCarrier='DL', norm=0.35),
 Row(UniqueCarrier='UA', norm=0.42),
 Row(UniqueCarrier='NK', norm=0.33),
 Row(UniqueCarrier='OO', norm=0.33),
 Row(UniqueCarrier='F9', norm=0.39),
 Row(UniqueCarrier='HA', norm=0.31),
 Row(UniqueCarrier='WN', norm=0.44)]

In [206]:
original = session.sql("""
SELECT UniqueCarrier, first(_c1), COUNT(*)
FROM Flights FF
INNER JOIN airlines AA
on FF.UniqueCarrier = AA._c3
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier;
""")
original.head(20)

[Row(UniqueCarrier='AA', first(_c1)='American Airlines', count(1)=23461),
 Row(UniqueCarrier='AS', first(_c1)='Alaska Airlines', count(1)=4488),
 Row(UniqueCarrier='B6', first(_c1)='JetBlue Airways', count(1)=9396),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count(1)=24334),
 Row(UniqueCarrier='EV', first(_c1)='Atlantic Southeast Airlines', count(1)=11596),
 Row(UniqueCarrier='F9', first(_c1)='Frontier Airlines', count(1)=2988),
 Row(UniqueCarrier='HA', first(_c1)='Hawaiian Airlines', count(1)=1939),
 Row(UniqueCarrier='NK', first(_c1)='Spirit Airlines', count(1)=4151),
 Row(UniqueCarrier='OO', first(_c1)='SkyWest', count(1)=16751),
 Row(UniqueCarrier='UA', first(_c1)='United Airlines', count(1)=17701),
 Row(UniqueCarrier='VX', first(_c1)='Virgin America', count(1)=2648),
 Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count(1)=47472)]

In [207]:
norms.createOrReplaceTempView("norms")
original.createOrReplaceTempView("original")

<h1 style="color: green">Flight Delays per Airline normalized</h1>

In [210]:
session.sql("""
SELECT o.UniqueCarrier, o.`first(_c1)`, n.norm 
FROM original o
INNER JOIN norms n
on o.UniqueCarrier = n.UniqueCarrier
""").head(20)

[Row(UniqueCarrier='AA', first(_c1)='American Airlines', norm=0.32),
 Row(UniqueCarrier='EV', first(_c1)='Atlantic Southeast Airlines', norm=0.33),
 Row(UniqueCarrier='B6', first(_c1)='JetBlue Airways', norm=0.38),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', norm=0.35),
 Row(UniqueCarrier='UA', first(_c1)='United Airlines', norm=0.42),
 Row(UniqueCarrier='NK', first(_c1)='Spirit Airlines', norm=0.33),
 Row(UniqueCarrier='OO', first(_c1)='SkyWest', norm=0.33),
 Row(UniqueCarrier='F9', first(_c1)='Frontier Airlines', norm=0.39),
 Row(UniqueCarrier='HA', first(_c1)='Hawaiian Airlines', norm=0.31),
 Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', norm=0.44),
 Row(UniqueCarrier='AS', first(_c1)='Alaska Airlines', norm=0.31),
 Row(UniqueCarrier='VX', first(_c1)='Virgin America', norm=0.46)]

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [211]:
result = %timeit -n5 -r5 -o sum(range(1000000))
### Write your code here
session.sql("""
SELECT UniqueCarrier, first(_c1), COUNT(*)
FROM Flights
INNER JOIN airlines
on Flights.UniqueCarrier = airlines._c3
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier;
""").head(20)

12.6 ms ± 941 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


[Row(UniqueCarrier='AA', first(_c1)='American Airlines', count(1)=23461),
 Row(UniqueCarrier='AS', first(_c1)='Alaska Airlines', count(1)=4488),
 Row(UniqueCarrier='B6', first(_c1)='JetBlue Airways', count(1)=9396),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count(1)=24334),
 Row(UniqueCarrier='EV', first(_c1)='Atlantic Southeast Airlines', count(1)=11596),
 Row(UniqueCarrier='F9', first(_c1)='Frontier Airlines', count(1)=2988),
 Row(UniqueCarrier='HA', first(_c1)='Hawaiian Airlines', count(1)=1939),
 Row(UniqueCarrier='NK', first(_c1)='Spirit Airlines', count(1)=4151),
 Row(UniqueCarrier='OO', first(_c1)='SkyWest', count(1)=16751),
 Row(UniqueCarrier='UA', first(_c1)='United Airlines', count(1)=17701),
 Row(UniqueCarrier='VX', first(_c1)='Virgin America', count(1)=2648),
 Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count(1)=47472)]

<h1 style="color: green">Average</h1>

In [212]:
result.average

0.012610774239874445

### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [79]:
### Write your code here


### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [80]:
### Write your code here
